In [ ]:
# Manejo de datos
import numpy as np
import pandas as pd
import os
from datetime import datetime

## Lectura del dataset

In [ ]:
# Cargar el archivo Excel
df = pd.read_excel("Datasets/accesos.xlsx")

# Leer días de vacaciones desde un archivo txt
with open('Datasets/dias_vacaciones.txt', 'r') as f:
    dias_vacaciones = [pd.to_datetime(line.strip(), errors='coerce') for line in f if line.strip()]

# Asegurar que la fecha esté en formato datetime
df['Fecha Completa'] = pd.to_datetime(df['Fecha Completa'])

# Crear variables temporales relevantes
df['Día'] = df['Fecha Completa'].dt.day
df['Mes'] = df['Fecha Completa'].dt.month
df['Año'] = df['Fecha Completa'].dt.year

# Crear una clave de fecha sin hora
df['Fecha'] = df['Fecha Completa'].dt.date

# Crear columna exógena de vacaciones
df['Vacaciones'] = df['Fecha Completa'].dt.normalize().isin(dias_vacaciones).astype(int)

# Agrupar y contar accesos
daily_counts = df.groupby(['Fecha', 'Día', 'Mes', 'Año']).agg({
    'Fecha Completa': 'count',  # esto cuenta los accesos
    'Vacaciones': 'max'  # max para saber si es día de vacaciones
}).rename(columns={'Fecha Completa': 'Accesos'}).reset_index()

# Crear rango completo de fechas
fecha_inicio = df['Fecha'].min()
fecha_fin = df['Fecha'].max()
fechas_completas = pd.DataFrame({'Fecha': pd.date_range(fecha_inicio, fecha_fin)})

# Extraer día, mes y año para el rango completo
fechas_completas['Día'] = fechas_completas['Fecha'].dt.day
fechas_completas['Mes'] = fechas_completas['Fecha'].dt.month
fechas_completas['Año'] = fechas_completas['Fecha'].dt.year
fechas_completas['Fecha'] = fechas_completas['Fecha'].dt.date 

# Marcar vacaciones en el rango completo
fechas_completas['Vacaciones'] = pd.to_datetime(fechas_completas['Fecha']).isin(dias_vacaciones).astype(int)

# Unir con daily_counts
daily_counts_full = fechas_completas.merge(
    daily_counts.drop(columns=['Vacaciones']),  # quitamos la columna para evitar duplicados
    on=['Fecha', 'Día', 'Mes', 'Año'],
    how='left'
)